In [ ]:
#ttest method for 1st and 2nd order, I have made it

def t_test(fixed_project,random_project,step=2000,order_2=False):
    no_samples = len(random_project[0])
#     step = no_samples
    tf = np.zeros(no_samples)
#     if(order_2):
    tf_2 = np.zeros(no_samples)
    # order_2= False

    for i in range(0,no_samples,step):
        stt = i;
        end= i+ step;
        if(end>no_samples):
            end=no_samples;
        #get traces for a no. of samples 
        r = []
        for i,trace in enumerate(random_project):
            r.append(trace[stt:end])
        f = []
        for i,tracef in enumerate(fixed_project):
            f.append(tracef[stt:end])

        #calculate t-test
        t = ttest_ind(f, r, axis=0, equal_var=False)[0] 

        #append the output
        tf[stt:end] = t

        if(order_2):
            #convert to numpy and preprocess them
            r= np.array(r)
            f = np.array(f)
            r_2 = (r-r.mean(axis=0))**2
            f_2 = (f-f.mean(axis=0))**2
            t_2 = ttest_ind(f_2,r_2,axis=0,equal_var=False)[0]
            tf_2[stt:end] = t_2
#         print("t-test calculated for [",stt,":",end,"]")

    return tf,tf_2;

In [ ]:
#DPA code for my workflow, this is just to give you an idea of what kind of codes are available and what we work with. 

#DPA calculation code adpated from chipwhisperer tutorials

#as we are not guessing any key but mainly going through the intermediate values for a specific key, 
#we are mainly interested in finding a correlation peak. Basically, we are looking for getting a correlation between non masked
#values and the HG on masked power traces.


from tqdm import tnrange
maxcpa = [0] * 1
num_trace = 1000000
dpa_trace = np.array(IV_project.waves[0:num_trace])
# dpa_trace = dpa_trace[:,14000:15000]
# dpa_trace = P


# we don't need to redo the mean and std dev calculations 
# for each key guess
t_bar = mean(dpa_trace) 
o_t = std_dev(dpa_trace, t_bar)
kguess = 0 # for us the key guess doesn't matter as we are not guessing 


# for kguess in tnrange(0, 256):
#     hws = np.array([[HW[aes_internal(textin[0],kguess)] for textin in textin_array]]).transpose()


hws = np.array([[intermediate_value(textout)for textout in dpa_project.textouts[0:num_trace]]]).transpose()
hws_bar = mean(hws)
o_hws = std_dev(hws, hws_bar)
correlation = cov(dpa_trace, t_bar, hws, hws_bar)
cpaoutput = correlation/(o_t*o_hws)
# maxcpa[kguess] = max(abs(cpaoutput))
maxcpa[kguess] = max(abs(cpaoutput))

#wrong key
# hws_not = np.array([[intermediate_value_not(textout)for textout in dpa_project.textouts[0:num_trace]]]).transpose()
# hws_bar_not = mean(hws_not)
# o_hws_not = std_dev(hws_not, hws_bar_not)
# correlation_not = cov(dpa_trace, t_bar, hws_not, hws_bar_not)
# cpaoutput_not = correlation/(o_t*o_hws_not)
# maxcpa[1] = max(abs(cpaoutput_not))

guess = np.argmax(maxcpa)
guess_corr = max(maxcpa)
# ###################
# END SOLUTION
# ###################
print("Key guess: ", hex(guess))
print("Correlation: ", guess_corr)
print(len(cpaoutput))